<a href="https://colab.research.google.com/github/parvathy3333/etp_prediction/blob/main/membrane%20fouling%20rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pickle
import base64
from io import BytesIO

# Set page configuration
st.set_page_config(
    page_title="Membrane Fouling Prediction Tool",
    page_icon="🧪",
    layout="wide"
)

# Page title and description
st.title("Membrane Fouling Prediction and Uncertainty Analysis")
st.subheader("Wastewater Treatment Plant Case Study")

# Sidebar with navigation
st.sidebar.title("Navigation")
page = st.sidebar.radio("Go to", ["Data Upload & Exploration", "Model Training", "Prediction", "Uncertainty Analysis"])

# Initialize session state variables if they don't exist
if 'data' not in st.session_state:
    st.session_state.data = None
if 'model' not in st.session_state:
    st.session_state.model = None
if 'X_train' not in st.session_state:
    st.session_state.X_train = None
if 'X_test' not in st.session_state:
    st.session_state.X_test = None
if 'y_train' not in st.session_state:
    st.session_state.y_train = None
if 'y_test' not in st.session_state:
    st.session_state.y_test = None
if 'scaler' not in st.session_state:
    st.session_state.scaler = None
if 'feature_names' not in st.session_state:
    st.session_state.feature_names = None

# Data Upload & Exploration page
if page == "Data Upload & Exploration":
    st.header("Data Upload & Exploration")

    # Data upload section
    st.subheader("Upload your wastewater treatment plant data")
    uploaded_file = st.file_uploader("Choose a CSV file", type="csv")

    # Example data option
    use_example_data = st.checkbox("Use example data instead")

    if use_example_data:
        # Generate example data for demonstration
        np.random.seed(42)
        n_samples = 1000

        # Define features that might affect membrane fouling
        data = {
            'pH': np.random.uniform(6.0, 8.5, n_samples),
            'MLSS_mg_L': np.random.uniform(2000, 5000, n_samples),  # Mixed liquor suspended solids
            'TMP_kPa': np.random.uniform(10, 50, n_samples),  # Transmembrane pressure
            'Flux_LMH': np.random.uniform(10, 30, n_samples),  # Permeate flux
            'Temperature_C': np.random.uniform(15, 30, n_samples),
            'COD_mg_L': np.random.uniform(200, 800, n_samples),  # Chemical oxygen demand
            'NH4_mg_L': np.random.uniform(10, 40, n_samples),  # Ammonium concentration
            'DO_mg_L': np.random.uniform(1, 5, n_samples),  # Dissolved oxygen
            'Conductivity_uS_cm': np.random.uniform(500, 2000, n_samples),
        }

        # Generate fouling rate based on features (simplified model)
        fouling_rate = (
            0.05 * data['TMP_kPa'] +
            0.02 * data['Flux_LMH'] +
            -0.1 * data['DO_mg_L'] +
            0.001 * data['MLSS_mg_L'] +
            0.005 * data['COD_mg_L'] +
            np.random.normal(0, 0.5, n_samples)  # Add some noise
        )

        data['Fouling_Rate'] = fouling_rate
        example_df = pd.DataFrame(data)
        st.session_state.data = example_df

    elif uploaded_file is not None:
        # Read the uploaded file
        try:
            data = pd.read_csv(uploaded_file)
            st.session_state.data = data
            st.success("Data successfully loaded!")
        except Exception as e:
            st.error(f"Error: {e}")

    # Display and explore the data if available
    if st.session_state.data is not None:
        st.subheader("Data Preview")
        st.dataframe(st.session_state.data.head())

        st.subheader("Data Information")
        buffer = BytesIO()
        st.session_state.data.info(buf=buffer)
        s = buffer.getvalue().decode()
        st.text(s)

        st.subheader("Statistical Summary")
        st.dataframe(st.session_state.data.describe())

        st.subheader("Data Visualization")

        # Correlation heatmap
        st.write("#### Correlation Heatmap")
        fig, ax = plt.subplots(figsize=(10, 8))
        corr = st.session_state.data.corr()
        sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", ax=ax)
        st.pyplot(fig)

        # Distribution of target variable
        st.write("#### Distribution of Fouling Rate")
        if 'Fouling_Rate' in st.session_state.data.columns:
            fig, ax = plt.subplots(figsize=(10, 6))
            sns.histplot(st.session_state.data['Fouling_Rate'], kde=True, ax=ax)
            st.pyplot(fig)
        else:
            st.warning("No 'Fouling_Rate' column found. Please ensure your data includes the target variable.")

        # Feature histograms
        st.write("#### Feature Distributions")
        selected_features = st.multiselect(
            "Select features to visualize",
            options=st.session_state.data.columns.tolist(),
            default=st.session_state.data.columns.tolist()[:3]
        )

        if selected_features:
            fig, axes = plt.subplots(len(selected_features), 1, figsize=(10, 3*len(selected_features)))
            if len(selected_features) == 1:
                axes = [axes]

            for i, feature in enumerate(selected_features):
                sns.histplot(st.session_state.data[feature], kde=True, ax=axes[i])
                axes[i].set_title(f"Distribution of {feature}")

            plt.tight_layout()
            st.pyplot(fig)

        # Select target variable
        st.subheader("Select Target Variable")
        target_col = st.selectbox(
            "Which column represents the membrane fouling rate?",
            options=st.session_state.data.columns.tolist(),
            index=st.session_state.data.columns.tolist().index('Fouling_Rate') if 'Fouling_Rate' in st.session_state.data.columns else 0
        )

        # Select features to use
        st.subheader("Select Features")
        feature_cols = st.multiselect(
            "Which columns would you like to use as features?",
            options=[col for col in st.session_state.data.columns if col != target_col],
            default=[col for col in st.session_state.data.columns if col != target_col]
        )

        if st.button("Prepare Data for Model Training"):
            if feature_cols and target_col:
                st.session_state.feature_names = feature_cols
                st.session_state.target_name = target_col
                st.success("Data prepared for model training! Please go to the Model Training page.")
            else:
                st.error("Please select both features and target variable.")


# Model Training page
elif page == "Model Training":
    st.header("Model Training")

    if st.session_state.data is None:
        st.warning("Please upload or generate data first on the Data Upload & Exploration page.")
    elif not hasattr(st.session_state, 'feature_names') or not st.session_state.feature_names:
        st.warning("Please select features and target variable on the Data Upload & Exploration page.")
    else:
        # Show selected features and target
        st.subheader("Selected Features and Target")
        st.write(f"Target Variable: {st.session_state.target_name}")
        st.write(f"Selected Features: {', '.join(st.session_state.feature_names)}")

        # Model configuration
        st.subheader("Model Configuration")

        col1, col2 = st.columns(2)

        with col1:
            test_size = st.slider("Test Set Size (%)", min_value=10, max_value=50, value=20, step=5) / 100
            n_estimators = st.slider("Number of Trees", min_value=50, max_value=500, value=100, step=50)

        with col2:
            max_depth = st.slider("Maximum Tree Depth", min_value=3, max_value=30, value=10, step=1)
            random_state = st.slider("Random State", min_value=0, max_value=100, value=42, step=1)

        # Train model button
        if st.button("Train Random Forest Model"):
            with st.spinner("Training model..."):
                try:
                    # Prepare data
                    X = st.session_state.data[st.session_state.feature_names]
                    y = st.session_state.data[st.session_state.target_name]

                    # Split data
                    X_train, X_test, y_train, y_test = train_test_split(
                        X, y, test_size=test_size, random_state=random_state
                    )

                    # Scale features
                    scaler = StandardScaler()
                    X_train_scaled = scaler.fit_transform(X_train)
                    X_test_scaled = scaler.transform(X_test)

                    # Train Random Forest model
                    model = RandomForestRegressor(
                        n_estimators=n_estimators,
                        max_depth=max_depth,
                        random_state=random_state,
                        n_jobs=-1
                    )

                    model.fit(X_train_scaled, y_train)

                    # Save the model and data
                    st.session_state.model = model
                    st.session_state.X_train = X_train
                    st.session_state.X_test = X_test
                    st.session_state.y_train = y_train
                    st.session_state.y_test = y_test
                    st.session_state.scaler = scaler

                    # Make predictions
                    y_pred = model.predict(X_test_scaled)

                    # Calculate metrics
                    r2 = r2_score(y_test, y_pred)
                    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                    mae = mean_absolute_error(y_test, y_pred)

                    # Display metrics
                    st.subheader("Model Performance Metrics")

                    metric_col1, metric_col2, metric_col3 = st.columns(3)

                    with metric_col1:
                        st.metric("R² Score", f"{r2:.4f}")

                    with metric_col2:
                        st.metric("RMSE", f"{rmse:.4f}")

                    with metric_col3:
                        st.metric("MAE", f"{mae:.4f}")

                    # Feature importance
                    st.subheader("Feature Importance")
                    importance = model.feature_importances_
                    indices = np.argsort(importance)[::-1]

                    fig, ax = plt.subplots(figsize=(10, 6))
                    feature_names = np.array(st.session_state.feature_names)
                    sns.barplot(x=importance[indices], y=feature_names[indices], ax=ax)
                    ax.set_title("Feature Importance")
                    ax.set_xlabel("Importance")
                    ax.set_ylabel("Feature")
                    st.pyplot(fig)

                    # Plot actual vs predicted values
                    st.subheader("Actual vs. Predicted Values")
                    fig, ax = plt.subplots(figsize=(10, 6))
                    ax.scatter(y_test, y_pred, alpha=0.5)
                    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
                    ax.set_xlabel("Actual Values")
                    ax.set_ylabel("Predicted Values")
                    ax.set_title("Actual vs. Predicted Values")
                    st.pyplot(fig)

                    # Save model button
                    st.subheader("Save Model")

                    def get_model_download_link(model):
                        output = BytesIO()
                        pickle.dump(model, output)
                        b64 = base64.b64encode(output.getvalue()).decode()
                        return f'<a href="data:application/octet-stream;base64,{b64}" download="membrane_fouling_model.pkl">Download trained model</a>'

                    st.markdown(get_model_download_link(model), unsafe_allow_html=True)

                    st.success("Model trained successfully! You can now use it to make predictions or analyze uncertainty.")

                except Exception as e:
                    st.error(f"Error during model training: {e}")

# Prediction page
elif page == "Prediction":
    st.header("Membrane Fouling Prediction")

    if st.session_state.model is None:
        st.warning("Please train a model first on the Model Training page.")
    else:
        st.subheader("Enter Parameter Values for Prediction")

        # Create input fields for each feature
        input_values = {}

        # Create columns for better layout
        col1, col2 = st.columns(2)

        # Add input fields for all features
        for i, feature in enumerate(st.session_state.feature_names):
            # Get min and max values from training data
            min_val = float(st.session_state.X_train[feature].min())
            max_val = float(st.session_state.X_train[feature].max())
            mean_val = float(st.session_state.X_train[feature].mean())

            # Create input slider in appropriate column
            if i % 2 == 0:
                with col1:
                    input_values[feature] = st.slider(
                        f"{feature}",
                        min_value=min_val,
                        max_value=max_val,
                        value=mean_val,
                        step=(max_val - min_val) / 100
                    )
            else:
                with col2:
                    input_values[feature] = st.slider(
                        f"{feature}",
                        min_value=min_val,
                        max_value=max_val,
                        value=mean_val,
                        step=(max_val - min_val) / 100
                    )

        # Add a button to make prediction
        if st.button("Predict Fouling Rate"):
            with st.spinner("Making prediction..."):
                try:
                    # Convert input to dataframe
                    input_df = pd.DataFrame([input_values])

                    # Scale input data
                    input_scaled = st.session_state.scaler.transform(input_df)

                    # Make prediction
                    prediction = st.session_state.model.predict(input_scaled)

                    # For uncertainty, get predictions from individual trees
                    tree_preds = np.array([tree.predict(input_scaled)[0] for tree in st.session_state.model.estimators_])

                    # Calculate uncertainty metrics
                    mean_pred = np.mean(tree_preds)
                    std_pred = np.std(tree_preds)
                    conf_interval = (mean_pred - 1.96 * std_pred, mean_pred + 1.96 * std_pred)

                    # Display prediction
                    st.subheader("Prediction Results")

                    col1, col2, col3 = st.columns(3)

                    with col1:
                        st.metric("Predicted Fouling Rate", f"{prediction[0]:.4f}")

                    with col2:
                        st.metric("Standard Deviation", f"{std_pred:.4f}")

                    with col3:
                        st.metric("Coefficient of Variation", f"{(std_pred/mean_pred)*100:.2f}%")

                    st.write(f"95% Confidence Interval: [{conf_interval[0]:.4f}, {conf_interval[1]:.4f}]")

                    # Visualize the prediction distribution
                    st.subheader("Prediction Distribution from Random Forest Trees")

                    fig, ax = plt.subplots(figsize=(10, 6))
                    sns.histplot(tree_preds, kde=True, ax=ax)
                    ax.axvline(prediction[0], color='red', linestyle='--', label='Prediction')
                    ax.axvline(conf_interval[0], color='green', linestyle='--', label='95% CI Lower')
                    ax.axvline(conf_interval[1], color='green', linestyle='--', label='95% CI Upper')
                    ax.set_xlabel("Predicted Fouling Rate")
                    ax.set_ylabel("Frequency")
                    ax.legend()
                    st.pyplot(fig)

                    # Contextual interpretation
                    st.subheader("Interpretation")

                    if std_pred / mean_pred < 0.1:
                        confidence = "high"
                    elif std_pred / mean_pred < 0.2:
                        confidence = "moderate"
                    else:
                        confidence = "low"

                    st.write(f"The model predicts a fouling rate of {prediction[0]:.4f} with {confidence} confidence.")
                    st.write(f"The standard deviation of {std_pred:.4f} represents the model's uncertainty.")

                    # Display what-if scenarios
                    st.subheader("What-If Analysis")
                    st.write("How would changing parameter values affect the fouling rate?")

                    # Get top 3 important features
                    importance = st.session_state.model.feature_importances_
                    top_features_idx = np.argsort(importance)[::-1][:3]
                    top_features = [st.session_state.feature_names[i] for i in top_features_idx]

                    selected_feature = st.selectbox(
                        "Select a parameter to vary:",
                        options=top_features
                    )

                    if selected_feature:
                        # Create range of values for the selected feature
                        min_val = float(st.session_state.X_train[selected_feature].min())
                        max_val = float(st.session_state.X_train[selected_feature].max())

                        # Generate predictions across the range
                        test_values = np.linspace(min_val, max_val, 50)
                        predictions = []
                        lower_bounds = []
                        upper_bounds = []

                        for val in test_values:
                            # Copy the input values and change only the selected feature
                            test_input = input_values.copy()
                            test_input[selected_feature] = val

                            # Convert to DataFrame
                            test_df = pd.DataFrame([test_input])

                            # Scale
                            test_scaled = st.session_state.scaler.transform(test_df)

                            # Get predictions from all trees
                            tree_preds = np.array([tree.predict(test_scaled)[0] for tree in st.session_state.model.estimators_])

                            # Calculate mean and CI
                            mean_pred = np.mean(tree_preds)
                            std_pred = np.std(tree_preds)

                            predictions.append(mean_pred)
                            lower_bounds.append(mean_pred - 1.96 * std_pred)
                            upper_bounds.append(mean_pred + 1.96 * std_pred)

                        # Plot what-if analysis
                        fig, ax = plt.subplots(figsize=(10, 6))
                        ax.plot(test_values, predictions, 'b-', label='Predicted Fouling Rate')
                        ax.fill_between(test_values, lower_bounds, upper_bounds, alpha=0.3, label='95% CI')
                        ax.set_xlabel(selected_feature)
                        ax.set_ylabel('Predicted Fouling Rate')
                        ax.set_title(f'Effect of {selected_feature} on Fouling Rate')
                        ax.axvline(input_values[selected_feature], color='red', linestyle='--', label='Current Value')
                        ax.legend()
                        st.pyplot(fig)

                        # Interpretation of what-if analysis
                        slope = (predictions[-1] - predictions[0]) / (test_values[-1] - test_values[0])
                        if abs(slope) < 0.01:
                            impact = "minimal impact"
                        elif abs(slope) < 0.1:
                            impact = "moderate impact"
                        else:
                            impact = "significant impact"

                        direction = "increase" if slope > 0 else "decrease"

                        st.write(f"Changing {selected_feature} has a {impact} on the fouling rate.")
                        st.write(f"As {selected_feature} increases, the fouling rate tends to {direction}.")

                except Exception as e:
                    st.error(f"Error during prediction: {e}")

# Uncertainty Analysis page
elif page == "Uncertainty Analysis":
    st.header("Uncertainty Analysis")

    if st.session_state.model is None:
        st.warning("Please train a model first on the Model Training page.")
    else:
        st.subheader("Model Uncertainty Analysis")

        # Generate predictions for test set
        X_test_scaled = st.session_state.scaler.transform(st.session_state.X_test)
        y_pred = st.session_state.model.predict(X_test_scaled)

        # Get predictions from individual trees for uncertainty estimation
        tree_preds = np.array([tree.predict(X_test_scaled) for tree in st.session_state.model.estimators_])

        # Calculate standard deviation across trees for each test point
        y_std = np.std(tree_preds, axis=0)

        # Calculate confidence intervals
        y_lower = y_pred - 1.96 * y_std
        y_upper = y_pred + 1.96 * y_std

        # Calculate if actual is within CI
        within_ci = (st.session_state.y_test >= y_lower) & (st.session_state.y_test <= y_upper)
        ci_coverage = np.mean(within_ci) * 100

        # Display CI coverage
        st.metric("95% CI Coverage", f"{ci_coverage:.2f}%",
                 delta=f"{ci_coverage - 95:.2f}%" if ci_coverage != 95 else None)

        # Plot uncertainty vs. error
        st.subheader("Uncertainty vs. Error Analysis")

        abs_error = np.abs(st.session_state.y_test.values - y_pred)

        fig, ax = plt.subplots(figsize=(10, 6))
        ax.scatter(y_std, abs_error, alpha=0.5)
        ax.set_xlabel("Prediction Standard Deviation (Uncertainty)")
        ax.set_ylabel("Absolute Prediction Error")
        ax.set_title("Relationship Between Model Uncertainty and Error")

        # Add correlation line
        m, b = np.polyfit(y_std, abs_error, 1)
        ax.plot(y_std, m*y_std + b, 'r-', label=f'Correlation: {np.corrcoef(y_std, abs_error)[0,1]:.2f}')
        ax.legend()

        st.pyplot(fig)

        # Comment on uncertainty-error relationship
        corr = np.corrcoef(y_std, abs_error)[0,1]
        if corr > 0.5:
            st.write("There is a strong positive correlation between model uncertainty and prediction error, "
                    "indicating that the model's uncertainty estimates are reliable predictors of actual error.")
        elif corr > 0.3:
            st.write("There is a moderate positive correlation between model uncertainty and prediction error, "
                    "suggesting that the model's uncertainty estimates provide some useful information about prediction quality.")
        else:
            st.write("There is a weak correlation between model uncertainty and prediction error, "
                    "indicating that the model's uncertainty estimates may not reliably predict actual error.")

        # Plot sorted test predictions with CI
        st.subheader("Predictions with Confidence Intervals")

        # Sort by prediction value for better visualization
        sorted_indices = np.argsort(y_pred)
        y_test_sorted = st.session_state.y_test.values[sorted_indices]
        y_pred_sorted = y_pred[sorted_indices]
        y_lower_sorted = y_lower[sorted_indices]
        y_upper_sorted = y_upper[sorted_indices]

        # Plot only a subset of points for clarity
        sample_size = min(100, len(y_pred))
        step = len(y_pred) // sample_size

        fig, ax = plt.subplots(figsize=(12, 6))
        x_indices = np.arange(len(y_pred_sorted))[::step]

        # Plot actual values
        ax.scatter(x_indices, y_test_sorted[::step], alpha=0.6, color='green', label='Actual')

        # Plot predictions with CI
        ax.errorbar(x_indices, y_pred_sorted[::step],
                   yerr=[(y_pred_sorted[::step] - y_lower_sorted[::step]),
                         (y_upper_sorted[::step] - y_pred_sorted[::step])],
                   fmt='o', alpha=0.6, color='blue', label='Predicted with 95% CI')

        ax.set_xlabel("Sample Index")
        ax.set_ylabel("Fouling Rate")
        ax.set_title("Predictions with Confidence Intervals vs. Actual Values")
        ax.legend()

        st.pyplot(fig)

        # Uncertainty distribution
        st.subheader("Uncertainty Distribution")

        fig, ax = plt.subplots(figsize=(10, 6))
        sns.histplot(y_std, kde=True, ax=ax)
        ax.set_xlabel("Prediction Standard Deviation")
        ax.set_ylabel("Frequency")
        ax.set_title("Distribution of Model Uncertainty")

        st.pyplot(fig)

        # Calibration analysis
        st.subheader("Uncertainty Calibration Analysis")

        # Create uncertainty bins
        n_bins = 10
        uncertainty_bins = np.linspace(y_std.min(), y_std.max(), n_bins + 1)
        bin_indices = np.digitize(y_std, uncertainty_bins) - 1

        bin_errors = []
        bin_uncertainties = []

        for i in range(n_bins):
            mask = bin_indices == i
            if np.sum(mask) > 0:
                bin_errors.append(np.mean(abs_error[mask]))
                bin_uncertainties.append(np.mean(y_std[mask]))

        if bin_errors and bin_uncertainties:
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.plot(bin_uncertainties, bin_errors, 'o-')
            ax.plot([0, max(bin_uncertainties)], [0, max(bin_uncertainties)], 'r--', label='Ideal Calibration')
            ax.set_xlabel("Mean Predicted Uncertainty")
            ax.set_ylabel("Mean Observed Error")
            ax.set_title("Uncertainty Calibration Plot")
            ax.legend()

            st.pyplot(fig)

            # Interpret calibration plot
            ratio = np.mean(bin_errors) / np.mean(bin_uncertainties) if np.mean(bin_uncertainties) > 0 else 0

            if 0.8 <= ratio <= 1.2:
                st.write("The model's uncertainty estimates are well-calibrated. The predicted uncertainty levels align closely with observed errors.")
            elif ratio < 0.8:
                st.write("The model tends to overestimate uncertainty. Actual errors are generally smaller than the model's uncertainty estimates.")
            else:
                st.write("The model tends to underestimate uncertainty. Actual errors are generally larger than the model's uncertainty estimates.")

        # Risk assessment
        st.subheader("Risk Assessment for Decision Making")

        # Calculate risk metrics
        high_uncertainty_threshold = np.percentile(y_std, 75)
        high_uncertainty_mask = y_std > high_uncertainty_threshold

        high_error_threshold = np.percentile(abs_error, 75)
        high_error_mask = abs_error > high_error_threshold

        true_positive_rate = np.sum(high_uncertainty_mask & high_error_mask) / np.sum(high_error_mask) if np.sum(high_error_mask) > 0 else 0

        st.write(f"When the model shows high uncertainty (top 25%), it correctly identifies {true_positive_rate*100:.1f}% of high-error predictions.")

        if true_positive_rate > 0.7:
            st.write("This suggests the uncertainty estimates are reliable indicators of prediction quality.")
        elif true_positive_rate > 0.5:
            st.write("The uncertainty estimates provide moderate value for identifying potential prediction failures.")
        else:
            st.write("The uncertainty estimates may not be reliable indicators of when the model will make large errors.")

        # Feature uncertainty contribution
        st.subheader("Feature Contribution to Uncertainty")

        # Create a simple linear model to assess feature contribution to uncertainty
        from sklearn.linear_model import LinearRegression

        lr_model = LinearRegression()
        lr_model.fit(st.session_state.X_test, y_std)

        # Get coefficients
        feature_uncertainty_coef = {feature: coef for feature, coef in zip(st.session_state.feature
                                                                           # Get coefficients
        feature_uncertainty_coef = {feature: coef for feature, coef in zip(st.session_state.feature_names, lr_model.coef_)}

        # Sort by absolute coefficient value
        sorted_uncertainty_features = sorted(feature_uncertainty_coef.items(), key=lambda x: abs(x[1]), reverse=True)

        # Plot feature contributions to uncertainty
        fig, ax = plt.subplots(figsize=(10, 6))
        features = [x[0] for x in sorted_uncertainty_features]
        coefs = [x[1] for x in sorted_uncertainty_features]

        colors = ['red' if c < 0 else 'blue' for c in coefs]
        sns.barplot(x=np.abs(coefs), y=features, palette=colors, ax=ax)
        ax.set_xlabel("Absolute Coefficient Value")
        ax.set_ylabel("Feature")
        ax.set_title("Feature Contribution to Prediction Uncertainty")

        st.pyplot(fig)

        # Interpret feature uncertainty
        st.write("Features with higher values contribute more to model uncertainty.")
        st.write(f"The top uncertainty-contributing features are: {', '.join(features[:3])}")

        # Recommendations for reducing uncertainty
        st.subheader("Recommendations for Reducing Prediction Uncertainty")

        st.write("Based on the uncertainty analysis, consider the following recommendations:")
        st.write("1. Collect more data for operating conditions that show high uncertainty")
        st.write(f"2. Improve measurement precision for {', '.join(features[:3])}")
        st.write("3. Consider adding additional sensors or measurements that might better explain fouling behavior")
        st.write("4. For critical operational decisions, use the confidence intervals rather than point predictions")

        # Download uncertainty analysis report
        st.subheader("Download Analysis Report")

        def get_uncertainty_report():
            report = f"""
            # Membrane Fouling Prediction Uncertainty Report

            ## Model Performance
            - 95% Confidence Interval Coverage: {ci_coverage:.2f}%
            - Correlation between uncertainty and error: {corr:.2f}

            ## Key Uncertainty Findings
            - Top uncertainty-contributing features: {', '.join(features[:3])}
            - Uncertainty detection rate for high errors: {true_positive_rate*100:.1f}%

            ## Recommendations
            1. Collect more data for operating conditions that show high uncertainty
            2. Improve measurement precision for {', '.join(features[:3])}
            3. Consider adding additional sensors or measurements that might better explain fouling behavior
            4. For critical operational decisions, use the confidence intervals rather than point predictions
            """
            return report

        report_text = get_uncertainty_report()
        report_bytes = report_text.encode()
        b64 = base64.b64encode(report_bytes).decode()

        st.markdown(
            f'<a href="data:application/octet-stream;base64,{b64}" download="uncertainty_analysis_report.md">Download Uncertainty Analysis Report</a>',
            unsafe_allow_html=True
        )

# Instructions for Google Colab setup
st.sidebar.header("Setup Instructions")

if st.sidebar.checkbox("Show Setup Instructions"):
    st.sidebar.subheader("Google Colab Setup")
    st.sidebar.markdown("""
    1. Create a new Google Colab notebook
    2. Install required packages:
    ```python
    !pip install streamlit pyngrok
    ```
    3. Copy this entire code into a cell
    4. Save the code to a file:
    ```python
    with open('membrane_fouling_app.py', 'w') as f:
        f.write('''
    # Paste the entire code here
    ''')
    ```
    5. Run the app with ngrok:
    ```python
    from pyngrok import ngrok
    !streamlit run membrane_fouling_app.py --server.port=8501 &
    public_url = ngrok.connect(port=8501)
    print(f"Streamlit app URL: {public_url}")
    ```
    """)

    st.sidebar.subheader("Local Streamlit Setup")
    st.sidebar.markdown("""
    1. Save the code to a file named `membrane_fouling_app.py`
    2. Install required packages:
    ```
    pip install streamlit pandas numpy matplotlib seaborn scikit-learn
    ```
    3. Run the app:
    ```
    streamlit run membrane_fouling_app.py
    ```
    """)